In [188]:
from transformers import Wav2Vec2Config, Wav2Vec2ForSequenceClassification
import torch
import librosa
from torch.utils.data import DataLoader
from datasets import load_dataset
from datasets import Dataset
from tqdm import tqdm
import numpy as np
import evaluate

In [189]:
class KWS_dataset(torch.utils.data.Dataset):
    def __init__(self, input_data, output_data):
        self.input_data = input_data
        self.output_data = output_data
        
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, index):
        keyword = self.output_data[index]
        audio_features = self.input_data[index]
        # return audio_features, keyword
        return {'audio': audio_features,
                'keyword': keyword
               }

In [190]:
data_path = '../data/'

In [191]:
train_dataloader = torch.load('../data/en_splits_10_trainloader')
dev_dataloader = torch.load('../data/en_splits_10_devloader')
test_dataloader = torch.load('../data/en_splits_10_testloader')

In [192]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [193]:
wav2vec_model = Wav2Vec2ForSequenceClassification(Wav2Vec2Config(
    vocab_size=10,
    hidden_size=512,
    num_hidden_layers=2,
    num_attention_heads=2,
    conv_dim=[512,512,512],
    conv_kernel=[2,2,2], 
    conv_stride=[5,2,2],
    mask_time_length=1,
    num_labels=10
))

In [194]:
wav2vec_model.to(device)
wav2vec_model.double()
wav2vec_model.train()

optim = torch.optim.Adam(wav2vec_model.parameters(), lr=5e-5)

In [196]:
metric = evaluate.load("accuracy")

In [197]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="wav2vec",
    
    # track hyperparameters and run metadata
    config={
    "architecture": "wav2vec2",
    "dataset": "en_10_1600",
    "epochs": 10,
    'hidden_size': 512,
    'num_hidden_layers': 2,
    'num_attention_heads': 2,
    'conv_dim': [512,512,512],
    'conv_kernel': [2,2,2], 
    'conv_stride': [5,2,2],
    'mask_time_length': 1,
    }
)

# simulate training
epochs = 10

for epoch in range(epochs):
    wav2vec_model.train()
    for batch in tqdm(train_dataloader):
        optim.zero_grad()
        audio = batch['audio'].to(device)
        labels = batch['keyword'].to(device)
        outputs = wav2vec_model(audio, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optim.step()
        
    torch.save({
            'epoch': epoch,
            'model_state_dict': wav2vec_model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': loss,
            }, f'../model/wav2vec/epoch_{epoch+1}')
        
    wav2vec_model.eval()
        
    for batch in tqdm(dev_dataloader):
        audio = batch['audio'].to(device)
        labels = batch['keyword'].to(device)
        outputs = wav2vec_model(audio, labels=labels)
        
        metric.add_batch(predictions=outputs.logits.argmax(-1), references=labels)

    wandb.log({"acc": metric.compute()['accuracy'], "loss": loss})
    
wandb.finish()

acc,▃▁▇▆█▅
loss,█▁▇▇▄▁
acc,0.13124
loss,2.18014


100%|█████████████████████████████████████████████| 4/4 [00:20<00:00,  5.08s/it]


acc,▁▅▄▇▆▆█▇▇▇
loss,▆▆█▃▇▄▁▃▁▂
acc,0.32905
loss,1.3794
